In [11]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 430 bytes | 430.00 KiB/s, done.
From https://github.com/Usaid-Siddiqui/QG_Ijepa
   0da9c0e..92c8561  main       -> origin/main
Updating 0da9c0e..92c8561
Fast-forward
 train_pretrain.py | 7 ++++++-
 1 file changed, 6 insertions(+), 1 deletion(-)


In [9]:
# We need to decompress the data and store the in chunks optimal for the dataloader to read

from utils import unpack_h5
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_train.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_train.h5")
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_test.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qq_medium_test.h5")
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_finetune.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_finetune.h5")

Target file /content/qg_fast.h5 already exists. Skipping.
Unpacking 20000 samples to /content/qg_fast_test.h5...
Progress: 0/20000
Progress: 5000/20000
Progress: 10000/20000
Progress: 15000/20000
Unpacking Complete.
Unpacking 20000 samples to /content/qg_fast_finetune.h5...
Progress: 0/20000
Progress: 5000/20000
Progress: 10000/20000
Progress: 15000/20000
Unpacking Complete.


In [3]:
!python -u train_pretrain.py

Traceback (most recent call last):
  File "/shared/usiddiqui/IJEPA/QG_Ijepa/train_pretrain.py", line 20, in <module>
    logger, checkpoint_dir, device = init_experiment(cfg)
                                     ~~~~~~~~~~~~~~~^^^^^
  File "/shared/usiddiqui/IJEPA/QG_Ijepa/utils/misc.py", line 61, in init_experiment
    os.makedirs(checkpoint_dir, exist_ok=True)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen os>", line 218, in makedirs
  File "<frozen os>", line 218, in makedirs
  File "<frozen os>", line 218, in makedirs
  [Previous line repeated 2 more times]
  File "<frozen os>", line 228, in makedirs
PermissionError: [Errno 13] Permission denied: '/content'


Time for finetuning and evaluation.

In [10]:
#!python train_finetune.py

--- Training MLP Head | Results path: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412 ---
Epoch 1: 100% 313/313 [01:32<00:00,  3.37it/s, acc=0.612, loss=0.644]
Epoch 2: 100% 313/313 [01:34<00:00,  3.32it/s, acc=0.638, loss=0.592]
Epoch 3: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.648, loss=0.636]
Epoch 4: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.65, loss=0.557]
--- Computing Final ROC AUC ---
100% 313/313 [01:34<00:00,  3.30it/s]
Figure(800x600)

Final AUC: 0.7074
Results saved to: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412
